In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd
from tqdm import tqdm
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage.filters import gaussian
from skimage import io
from PIL import Image

%matplotlib widget

In [ ]:
folder = 'collected'  # 'collected' or 'downloaded'
gt_file = pd.read_csv(f'../Data/{folder}/joint_data/gts_new.csv')
uc_data = pd.read_csv(f'../Data/ucdavis-data.csv')

In [3]:
master_df = pd.DataFrame(columns=["File Path", "y"])
length_df = pd.DataFrame(columns=["Length", "y"])

lbp_def = []
lbp_ror = []
lbp_uni = []
lbp_nri = []

lbp_def_10 = []
lbp_ror_10 = []
lbp_uni_10 = []
lbp_nri_10 = []

color_256 = pd.DataFrame(columns=[*list(range(256)), "y"])
smoothed_256 = pd.DataFrame(columns=[*list(range(256)), "y"])

In [ ]:
def lbp_hist(lbp, n_bins=None):
    n_bins = int(lbp.max() + 1) if n_bins is None else 10
    return np.histogram(lbp, bins=n_bins)

In [ ]:
for row in tqdm(gt_file.iterrows()):
    slab, is_replaced = row[1][0], row[1][1]
    master_df = pd.concat([master_df, pd.DataFrame([[slab, int(is_replaced)]], columns=master_df.columns)])

    img = io.imread(os.path.join("joint_data", "all", slab), as_gray=True)
    # length_df = pd.concat([length_df, pd.DataFrame([[img.shape[0], int(is_replaced)]], columns=length_df.columns)])
    length_df = pd.concat(length_df, uc_data[uc_data.filename == slab].elem_length.iloc[0].item(), columns=length_df.columns)
    
    smoothed_img = gaussian(img)

    lbp_img_def = local_binary_pattern(img, 8, 1, 'default')
    lbp_img_ror = local_binary_pattern(img, 8, 1, 'ror')
    lbp_img_uni = local_binary_pattern(img, 8, 1, 'uniform')
    lbp_img_nri = local_binary_pattern(img, 8, 1, 'nri_uniform')

    lbp_def.append([*lbp_hist(lbp_img_def)[0], int(is_replaced)])
    lbp_ror.append([*lbp_hist(lbp_img_ror)[0], int(is_replaced)])
    lbp_uni.append([*lbp_hist(lbp_img_uni)[0], int(is_replaced)])
    lbp_nri.append([*lbp_hist(lbp_img_nri)[0], int(is_replaced)])
    
    lbp_def_10.append([*lbp_hist(lbp_img_def, 10)[0], int(is_replaced)])
    lbp_ror_10.append([*lbp_hist(lbp_img_ror, 10)[0], int(is_replaced)])
    lbp_uni_10.append([*lbp_hist(lbp_img_uni, 10)[0], int(is_replaced)])
    lbp_nri_10.append([*lbp_hist(lbp_img_nri, 10)[0], int(is_replaced)])

    color_256 = pd.concat([color_256, pd.DataFrame([[*np.histogram(img, bins=256, range=(0,1))[0], int(is_replaced)]])])
    # color_128 = pd.concat([color_128, pd.DataFrame([[*np.histogram(img, bins=128, range=(0,1))[0], is_replaced]])])
    # color_64 = pd.concat([color_64, pd.DataFrame([[*np.histogram(img, bins=64, range=(0,1))[0], is_replaced]])])
    # color_32 = pd.concat([color_32, pd.DataFrame([[*np.histogram(img, bins=32, range=(0,1))[0], is_replaced]])])

    smoothed_256 = pd.concat([smoothed_256, pd.DataFrame([[*np.histogram(smoothed_img, bins=256, range=(0,1))[0], int(is_replaced)]])])
    # smoothed_128 = pd.concat([smoothed_128, pd.DataFrame([[*np.histogram(smoothed_img, bins=128, range=(0,1))[0], is_replaced]])])
    # smoothed_64 = pd.concat([smoothed_64, pd.DataFrame([[*np.histogram(smoothed_img, bins=64, range=(0,1))[0], is_replaced]])])
    # smoothed_32 = pd.concat([smoothed_32, pd.DataFrame([[*np.histogram(smoothed_img, bins=32, range=(0,1))[0], is_replaced]])])




In [ ]:
temp = pd.Series(lbp_ror.loc[0].where(lbp_ror.sum(axis=0) != 0).notnull())
temp = temp.index[temp == True].tolist()
temp = [int(t) for t in temp]
lbp_ror = lbp_ror.iloc[:,temp]

raw_256 = color_256.to_numpy()
smoothed_256 = smoothed_256.to_numpy()

raw_128 = [np.sum(raw_256[:, i: i + int(256 / 128)], axis=1) for i in range(0, 256, int(256 / 128))]
raw_128 = np.array(raw_128).transpose()

smoothed_128 = [np.sum(smoothed_256[:, i: i + int(256 / 128)], axis=1) for i in range(0, 256, int(256 / 128))]
smoothed_128 = np.array(smoothed_128).transpose()

raw_64 = [np.sum(raw_256[:, i: i + int(256 / 64)], axis=1) for i in range(0, 256, int(256 / 64))]
raw_64 = np.array(raw_64).transpose()

smoothed_64 = [np.sum(smoothed_256[:, i: i + int(256 / 64)], axis=1) for i in range(0, 256, int(256 / 64))]
smoothed_64 = np.array(smoothed_64).transpose()

raw_32 = [np.sum(raw_256[:, i: i + int(256 / 32)], axis=1) for i in range(0, 256, int(256 / 32))]
raw_32 = np.array(raw_32).transpose()

smoothed_32 = [np.sum(smoothed_256[:, i: i + int(256 / 32)], axis=1) for i in range(0, 256, int(256 / 32))]
smoothed_32 = np.array(smoothed_32).transpose()

In [ ]:
master_df.to_csv('master_df.csv')
length_df.to_csv("length_df.csv")

pd.DataFrame(lbp_def).to_csv('lbp_def_df.csv')
pd.DataFrame(lbp_ror).to_csv('lbp_ror_df.csv')
pd.DataFrame(lbp_uni).to_csv('lbp_uni_df.csv')
pd.DataFrame(lbp_nri).to_csv('lbp_nri_df.csv')
pd.DataFrame(lbp_def_10).to_csv('lbp_def_10_df.csv')
pd.DataFrame(lbp_ror_10).to_csv('lbp_ror_10_df.csv')
pd.DataFrame(lbp_uni_10).to_csv('lbp_uni_10_df.csv')
pd.DataFrame(lbp_nri_10).to_csv('lbp_nri_10_df.csv')

color_256.to_csv('color_256.csv')
smoothed_256.to_csv('smoothed_256.csv')
pd.DataFrame(raw_128).to_csv("color_128.csv")
pd.DataFrame(smoothed_128).to_csv("smoothed_128.csv")
pd.DataFrame(raw_64).to_csv('color_64.csv')
pd.DataFrame(smoothed_64).to_csv("smoothed_64.csv")
pd.DataFrame(raw_32).to_csv("color_32.csv")
pd.DataFrame(smoothed_32).to_csv("smoothed_32.csv")